<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2022/blob/main/NRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che4.ipynb
!pip install importnb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from importnb import Notebook
with Notebook(): 
    from che4 import Props, qtox, xtoq

import numpy as np
from scipy.optimize import root, brentq
from plotly.subplots import make_subplots
from scipy.special import expit

### NRTL Activity Coefficient Model

$ln (\gamma_i) = \dfrac{ \sum\limits_{j=1}^{n}x_j \tau_{ji}G_{ji}}{\sum\limits_{k=1}^{n}x_kG_{ki}} + 
\sum\limits_{j=1}^{n} \dfrac{x_jG_{ij}}{\sum\limits_{k=1}^{n}x_k G_{kj}} \left( \tau_{ij} - 
\dfrac{\sum\limits_{m=1}^{n} x_m \tau_{mj} G_{mj}}{\sum\limits_{k=1}^{n} x_kG_{kj}} \right)$

$\tau_{ji} = A_{ji} + \dfrac{B_{ji}}{T} + C_{ji}ln(T)+D_{ji}T$

$G_{ji}=exp \left( - \alpha_{ji} \tau_{ji} \right)$

$\alpha_{ij}=\alpha_{ji}$

In [3]:
def NRTL_gamma_loop(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    lng = np.zeros_like(x)
    xtauGdivxG = np.zeros_like(x)
    xG = np.zeros_like(x)
    term2 = np.zeros_like(x)
    N = x.size
    for i in range(N):
        xtauG = 0.
        for j in range(N):
            xtauG += x[j]*tau[j,i]*G[j,i]
            xG[i] += x[j]*G[j,i]
        xtauGdivxG[i] = xtauG/xG[i]
    for i in range(N):
        for j in range(N):
            term2[i]+= x[j]*G[i,j]/xG[j]*(tau[i,j] - xtauGdivxG[j])

    return np.exp(xtauGdivxG + term2)

def NRTL_gamma_einsum(x, T):
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = np.einsum('j,ji->i',x,tau*G)
    xG = np.einsum('j,ji',x,G)
    xtauGdivxG=xtauG/xG
    term2=np.einsum('j,ij->i',x,G*(tau-xtauGdivxG[None,:])/xG[None,:] )
    return np.exp(xtauGdivxG + term2)

def NRTL_gamma_matvec(x,T):
    x=x.reshape((1,-1))
    tau = p.NRTL_B/T
    G = np.exp(-p.NRTL_alpha*tau)
    xtauG = x@ (tau*G)
    xG = x@G
    xtauGdivxG=xtauG/xG
    a=G*(tau-xtauGdivxG[None,:])/xG[None,:]
    lng= xtauGdivxG + a@(x.squeeze())
    return np.exp(lng.flatten())

In [4]:
p = Props(['Ethanol','Water'])

In [5]:
def bubblePy_NRTL(x, T):
    Pi=x*NRTL_gamma_matvec(x,T)*p.Pvap(T)
    P = np.sum(Pi)
    return P, Pi/P 

def bubbleT_NRTL(x, P):
    Tb=p.Tb(P)
    Tlo, Thi = np.min(Tb), np.max(Tb)

    def eq(T):
        return bubblePy_NRTL(x,T)[0]-P

    T = brentq(eq, Tlo-40, Thi+40)

    return T, x*p.Pvap(T)/P 

def dewP_ideal(y, T):
    pvap = p.Pvap(T)
    P =  1/(np.sum(y/pvap))
    x = y * P/pvap
    return P, x

def dewT_ideal(y, P):
    Tb=p.Tb(P)
    Tlo, Thi = np.min(Tb), np.max(Tb)

    def eq(T):
        return dewP_ideal(y,T)[0]-P

    T = brentq(eq, Tlo-40, Thi+40)
    return T,  y*P/p.Pvap(T)


def dewP_NRTL(y, T):

    def eq(v):
        x = qtox(v[:-1])
        P = v[-1]
        return x*NRTL_gamma_matvec(x,T)*p.Pvap(T)-y*P

    P_ideal, x_ideal = dewP_ideal(y,T)
    v_guess=np.append(xtoq(x_ideal),P_ideal)
    sol=root(eq,v_guess).x
    return sol[-1], qtox(sol[:-1])

def dewT_NRTL(y, P):
    def eq(v):
        x = qtox(v[:-1])
        T = v[-1]
        return x*NRTL_gamma_matvec(x,T)*p.Pvap(T)-y*P

    T_ideal, x_ideal = dewT_ideal(y,P)
    v_guess=np.append(xtoq(x_ideal),T_ideal)
    sol=root(eq,v_guess).x
    return sol[-1], qtox(sol[:-1]), 



In [6]:
z1=np.linspace(0,1,101)
zs=np.c_[z1, 1-z1]

In [7]:
P=101325
bubbleT_list=[]
bubbley1_list=[]
for z in zs:
    bubbleT, y_bubble = bubbleT_NRTL(z, P)
    bubbleT_list.append(bubbleT)
    bubbley1_list.append(y_bubble[0])



In [8]:
fig=make_subplots()
fig.add_scatter(x=bubbley1_list,y=bubbleT_list,mode='lines')
fig.add_scatter(x=z1,y=bubbleT_list, mode='lines')
fig.update_layout(width=800, height=800, template='plotly_dark')

In [9]:
p = Props(['Methanol','Ethanol','Water'])

In [10]:
nF = np.array([10, 10, 10])
flashP = 101325

In [11]:
zF = nF/np.sum(nF)

In [12]:
bubbleT, _ = bubbleT_NRTL(zF, flashP)
dewT, _ = dewT_NRTL(zF, flashP)

In [13]:
flashT=np.mean([bubbleT, dewT])

In [46]:
def flashTP(nF, T, P):

    def fug_eq(v):
        nV = nF*expit(v)
        nL = nF - nV
        x = nL/np.sum(nL)
        y = nV/np.sum(nV)
        return x*NRTL_gamma_matvec(x,T)*p.Pvap(T) - y*P
        
    nV = nF*expit(root(fug_eq, np.zeros(3)).x)
    nL = nF-nV
    x = nL/np.sum(nL)
    y = nV/np.sum(nV)

    return x, y, np.sum(nV)/np.sum(nF)

In [17]:
flashTP(nF, flashT, flashP)

(array([0.24136541, 0.29673509, 0.4618995 ]),
 array([0.38461874, 0.35374214, 0.26163912]),
 0.6419950224372866)

In [30]:
#Adiabatic Flash Calculation
nFeed = np.array([10, 10, 10])
zF = nFeed/np.sum(nFeed)
FeedP = 3e5
FlashP = 1e5
FeedT, _ = bubbleT_NRTL(zF, FeedP)   #Feed is saturated liquid
FeedH = p.Hl(nFeed, FeedT)

In [ ]:
def flashPQ(nF, flashP, Q):
    

In [53]:
def H_balance(T):
    x,y, fracV = flashTP(nF, T, FlashP)
    nV = y*fracV*np.sum(nFeed)
    nL = x*(1-fracV)*np.sum(nFeed)
    return [p.Hv(nV, T) + p.Hl(nL, T) - FeedH]

In [55]:
res=root(H_balance, [346])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in true_divide



In [57]:
flashTP(nF,res.x[0],FlashP)

(array([0.32036409, 0.33335227, 0.34628364]),
 array([0.46849883, 0.33313593, 0.19836523]),
 0.0875503096267397)